In [ ]:
!pip install tensorflow-gpu==2.3.0

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Apple dataset/Train'
valid_path = '/content/drive/MyDrive/Apple dataset/Test'


In [ ]:
# Import the ResNet50 library as shown below and add preprocessing layer to the front of ResNet
# Here we will be using imagenet weights

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [ ]:
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
  # useful for getting number of output classes
folders = glob('/content/drive/MyDrive/Apple dataset/Train/*')
print(len(folders))


3


In [ ]:
# our layers -we can add more if we wantx = Flatten()(resnet.output)
x = Flatten()(resnet.output)


In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)
# as there are 3 classes for predicting shelf life 
# that's why we use softmax as activtion function

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)



In [ ]:
# view the structure of the model
model.summary()


Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Data Augumentation
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Apple dataset/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')


Found 460 images belonging to 3 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Apple dataset/Test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 148 images belonging to 3 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=100,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/100
29/29 [==============================] - 12s 397ms/step - loss: 0.9009 - accuracy: 0.7022 - val_loss: 1.5743 - val_accuracy: 0.5743
Epoch 2/100
29/29 [==============================] - 9s 308ms/step - loss: 0.8378 - accuracy: 0.6674 - val_loss: 0.9202 - val_accuracy: 0.5878
Epoch 3/100
29/29 [==============================] - 9s 312ms/step - loss: 0.6394 - accuracy: 0.7261 - val_loss: 0.8325 - val_accuracy: 0.6757
Epoch 4/100
29/29 [==============================] - 9s 308ms/step - loss: 0.9461 - accuracy: 0.6413 - val_loss: 1.1690 - val_accuracy: 0.6757
Epoch 5/100
29/29 [==============================] - 9s 312ms/step - loss: 1.1760 - accuracy: 0.6413 - val_loss: 1.1373 - val_accuracy: 0.7162
Epoch 6/100
29/29 [==============================] - 9s 310ms/step - loss: 1.0616 - accuracy: 0.6217 - val_loss: 1.3996 - val_accuracy: 0.5000
Epoch 7/100
29/29 [==============================] - 9s 312ms/step - loss: 0.6518 - accuracy: 0.7391 - val_loss: 0.7140 - val_accuracy: 0.743

In [ ]:
from tensorflow.keras.models import load_model
path="/content/drive/MyDrive/model_ResNet(accuracy=79).h5"
model.save(path)


In [ ]:
y_pred=model.predict(test_set)

In [ ]:
y_pred

array([[1.56532121e-06, 9.99995947e-01, 2.48351330e-06],
       [6.58134639e-01, 2.03536734e-01, 1.38328627e-01],
       [3.08404607e-03, 5.64343203e-03, 9.91272509e-01],
       [2.63646007e-01, 4.40547317e-01, 2.95806736e-01],
       [2.34181091e-01, 3.78366172e-01, 3.87452692e-01],
       [1.28493886e-02, 9.77326810e-01, 9.82382149e-03],
       [6.02306902e-01, 2.90642842e-03, 3.94786686e-01],
       [2.85679311e-01, 6.11444294e-01, 1.02876335e-01],
       [2.97096938e-01, 5.98039508e-01, 1.04863591e-01],
       [1.25549696e-02, 9.79602993e-01, 7.84193724e-03],
       [2.43194148e-01, 5.55348635e-01, 2.01457217e-01],
       [6.55891793e-03, 4.68189895e-01, 5.25251150e-01],
       [2.45939106e-01, 3.83384468e-04, 7.53677547e-01],
       [5.66002131e-01, 4.33031842e-03, 4.29667532e-01],
       [2.71403207e-03, 4.26898198e-03, 9.93016958e-01],
       [7.04310238e-01, 1.90643303e-03, 2.93783367e-01],
       [2.05020204e-01, 7.78871588e-04, 7.94200957e-01],
       [1.55919909e-01, 5.67900

In [ ]:
import numpy as np
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
y_pred

array([1, 0, 2, 1, 2, 1, 0, 1, 1, 1, 1, 2, 2, 0, 2, 0, 2, 1, 2, 1, 1, 0,
       1, 0, 1, 2, 2, 1, 1, 2, 2, 1, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1,
       1, 1, 2, 2, 0, 1, 0, 1, 1, 2, 1, 2, 1, 2, 0, 0, 1, 2, 0, 1, 1, 1,
       2, 1, 1, 2, 1, 1, 0, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 0, 2, 1, 1,
       2, 1, 1, 2, 2, 1, 1, 0, 2, 1, 1, 2, 0, 1, 1, 2, 1, 0, 2, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 0, 1, 1, 2,
       2, 0, 2, 2, 2, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('/content/drive/MyDrive/model_ResNet(accuracy=79).h5')

In [ ]:
img=image.load_img('/content/drive/MyDrive/Apple dataset/Train/Apple(1-5)/frame1010.jpg',target_size=(224,224))

In [ ]:
x=image.img_to_array(img)
x

array([[[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.]],

       ...,

       [[222., 222., 222.],
        [222., 222., 222.],
        [222., 222., 222.],
        ...,
        [ 51.,  35.,  22.],
        [ 51.,  35.,  22.],
        [ 55.,  37.,  23.]],

       [[223., 223., 223.],
        [223., 223., 223.],
        [223., 223., 223.],
        ...,
        [ 28.,  26.,  14.],
        [ 28.,  26.,  14.],
        [ 30.,  26.,  15.]],

       [[222., 222., 222.],
        [222., 222., 222.],
        [222., 2

In [ ]:
x.shape

(224, 224, 3)

In [ ]:
x=x/255

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
model.predict(img_data)

array([[1.000000e+00, 0.000000e+00, 3.023413e-26]], dtype=float32)

In [ ]:
a=np.argmax(model.predict(img_data),axis=1)

In [ ]:
print(a[0])
if a[0]==0:
  print("predicted Shelf-life is about (1-5)")
elif a[0]==1:
  print("predicted Shelf life is about (10-14)")
else:
  print("predicted Shelf life is about (10-14)")


0
predicted Shelf-life is about (1-5)
